In [1]:
import pandas as pd
import numpy as np
import reduction_function as rfun
import polarization_function as pfun
import networkx as nx

/home/jnye/miniconda3/envs/bach_env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
net_file = pd.read_csv("output_network/network_filtered.csv", sep="\t")
G = nx.from_pandas_edgelist(net_file, "src", "trg")
print(G)

Graph with 511 nodes and 2785 edges


#### In the real scenario, we won't be reading network_users_combined.csv, we will be getting the reclustered labelled monthly messages on each topic and merging them in single df. 

#### After that we will have to apply averaging for the users who write more than 1 message for a single topic.

In [3]:
data = pd.read_csv("output_network/network_users_combined.csv", sep="\t")
data.head(), data.shape

(            index  nodeid
 0     novagenesis     473
 1         jscoppe    6978
 2           sje46    8280
 3  reconditecache    8339
 4         Delheru   14848,
 (511, 2))

In [4]:
#dataset for the other 2 topics
data2 = data.copy()
data3 = data.copy()

Each dataset holds the person's stance on a single topic

In [5]:
def simulate_data(data, topic):
    n_rows = data.shape[0]
    data["A"] = np.random.rand(n_rows)
    data["F"] = np.random.rand(n_rows)
    data["N"] = np.random.rand(n_rows)
    data[f"stance_{topic}"] = data.apply(lambda x: rfun.get_stance(x), axis = 1)
    return data

In [6]:
data1 = simulate_data(data, "lib")
data2 = simulate_data(data2, "abortion")
data3 = simulate_data(data3, "marriage")

In [7]:
data3.head()

,index,nodeid,A,F,N,stance_marriage
0,novagenesis,473,0.266770,0.099914,0.791531,0.266770
1,jscoppe,6978,0.787995,0.561829,0.024986,0.787995
2,sje46,8280,0.366464,0.661807,0.589773,-0.661807
3,reconditecache,8339,0.913703,0.657821,0.162408,0.913703
4,Delheru,14848,0.564534,0.361023,0.789909,0.564534


UMAP and PCA on the simulated data

In [8]:
# full_data = rfun.merge_datasets(data1, data2, data3) ##use this on the real data
merged_df = pd.merge(pd.merge(data1, data2, on=['nodeid', 'index']), data3, on=['nodeid', 'index'])

In [9]:
merged_df.head()

,index,nodeid,A_x,F_x,N_x,stance_lib,A_y,F_y,N_y,stance_abortion,A,F,N,stance_marriage
0,novagenesis,473,0.724725,0.483781,0.874938,0.724725,0.179796,0.167515,0.422956,0.179796,0.266770,0.099914,0.791531,0.266770
1,jscoppe,6978,0.119825,0.081019,0.177145,0.119825,0.365102,0.125894,0.868668,0.365102,0.787995,0.561829,0.024986,0.787995
2,sje46,8280,0.420671,0.393219,0.356383,0.420671,0.406841,0.420684,0.313966,-0.420684,0.366464,0.661807,0.589773,-0.661807
3,reconditecache,8339,0.746393,0.767681,0.612934,-0.767681,0.919221,0.210527,0.613461,0.919221,0.913703,0.657821,0.162408,0.913703
4,Delheru,14848,0.220932,0.015198,0.214876,0.220932,0.280885,0.665635,0.702569,-0.665635,0.564534,0.361023,0.789909,0.564534


In [10]:
merged_df_reduced = rfun.apply_pca(merged_df, ["stance_lib", "stance_abortion", "stance_marriage"])
merged_df_reduced = rfun.apply_umap(merged_df, ["stance_lib", "stance_abortion", "stance_marriage"])

/home/jnye/miniconda3/envs/bach_env/lib/python3.10/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/home/jnye/miniconda3/envs/bach_env/lib/python3.10/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


#### Saving the user scores in the format needed for the polaization code 

In [11]:
user_scores_df_pca = merged_df_reduced[["nodeid", "pca_component"]]
user_scores_df_pca

,nodeid,pca_component
0,473,-0.458463
1,6978,-0.424900
2,8280,-0.067715
3,8339,0.465945
4,14848,-1.288888
...,...,...
506,1236908,-0.830910
507,1237608,-0.183360
508,1237816,-0.422343
509,1238295,-1.954628


In [12]:
user_scores_df_umap = merged_df_reduced[["nodeid", "umap_component"]]
user_scores_df_umap

,nodeid,umap_component
0,473,-14.545551
1,6978,-15.232685
2,8280,-4.892288
3,8339,13.254095
4,14848,-8.938305
...,...,...
506,1236908,-5.967499
507,1237608,12.436734
508,1237816,-15.271129
509,1238295,-10.392038


In [13]:
# user_scores_df_umap.to_csv("output_network/user_scores_umap.csv", sep=",", index=False)

In [14]:
# user_scores_df_pca.to_csv("output_network/user_scores_pca.csv", sep=",", index=False)

##### Polarization code 

In [ ]:
pfun.calc_pol2("output_network/network_filtered.csv", "output_network/user_scores_umap.csv")

In [ ]:
pfun.calc_pol2("output_network/network_filtered.csv", "output_network/user_scores_umap.csv")

       src      trg  nij     score
0   798787   254684    1  0.977561
1    15548   430231   16  0.999147
2  1209510   410689   13  0.998674
3  1209510   695649    1  0.915443
4   419460  1234244    3  0.981948
Graph with 511 nodes and 2785 edges
[798787, 254684, 15548, 430231, 1209510, 410689, 695649, 419460, 1234244, 520690, 972375, 521147, 357231, 706029, 608682, 1195573, 494890, 146354, 1032836, 1132377, 1137789, 748633, 21729, 1220350, 1035996, 305666, 1230484, 1153602, 1232506, 1224593, 565896, 747924, 126762, 164809, 594034, 787767, 1008547, 530659, 377344, 958675, 611626, 253973, 1118102, 1137978, 387320, 1147333, 716630, 1170058, 91872, 551205, 871004, 909695, 741859, 1223152, 729200, 1211908, 973627, 1083511, 1208415, 1204545, 817123, 1214536, 1172804, 999706, 1210621, 166171, 246226, 1213277, 816708, 1225587, 1177038, 1212815, 1190564, 1193349, 1213020, 1024416, 413278, 928681, 422090, 1190023, 1051525, 190029, 1137417, 1201133, 198633, 646929, 641233, 942261, 1014902, 108015

np.float64(215.39403942350847)

In [16]:
pfun.calc_pol3("output_network/network_filtered.csv", "output_network/user_scores_umap.csv", "umap_component")

       src      trg  nij     score
0   798787   254684    1  0.977561
1    15548   430231   16  0.999147
2  1209510   410689   13  0.998674
3  1209510   695649    1  0.915443
4   419460  1234244    3  0.981948
Graph with 511 nodes and 2785 edges
[798787, 254684, 15548, 430231, 1209510, 410689, 695649, 419460, 1234244, 520690, 972375, 521147, 357231, 706029, 608682, 1195573, 494890, 146354, 1032836, 1132377, 1137789, 748633, 21729, 1220350, 1035996, 305666, 1230484, 1153602, 1232506, 1224593, 565896, 747924, 126762, 164809, 594034, 787767, 1008547, 530659, 377344, 958675, 611626, 253973, 1118102, 1137978, 387320, 1147333, 716630, 1170058, 91872, 551205, 871004, 909695, 741859, 1223152, 729200, 1211908, 973627, 1083511, 1208415, 1204545, 817123, 1214536, 1172804, 999706, 1210621, 166171, 246226, 1213277, 816708, 1225587, 1177038, 1212815, 1190564, 1193349, 1213020, 1024416, 413278, 928681, 422090, 1190023, 1051525, 190029, 1137417, 1201133, 198633, 646929, 641233, 942261, 1014902, 108015

np.float64(215.39403942350847)